In [ ]:
%pip install psycopg2-binary pandas sqlalchemy plotly ipywidgets ipympl nbformat ipython updatemenus

In [ ]:
%pip freeze > ./requirements.txt

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import locale

locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "global"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

custom_palette = px.colors.qualitative.D3

fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.4, 0.6],
    row_heights=[0.5, 0.5],
    specs=[
        [{"type": "table"}, {"type": "xy", "rowspan": 2}],
        [{"type": "table"}, None]
    ],
    horizontal_spacing=0.05,
    vertical_spacing=0.1
)

fig.add_trace(
    go.Table(
        header=dict(values=["Heure", "Humidité (%)", "Capteur"], fill_color="lightgrey", align="left"),
        cells=dict(values=[df.time_bucket, df.value, df.id_tag], fill_color="white", align="left")
    ),
    row=1, col=1
)

fig.add_trace(
    go.Table(
        header=dict(
            values=["Capteur", "Moyenne (%)", "Minimum (%)", "Maximum (%)", "Nombre de valeurs"],
            fill_color="lightgrey", align="left"
        ),
        cells=dict(
            values=[
                stats_complet["id_tag"],
                round(stats_complet["moyenne"], 2),
                round(stats_complet["minimum"], 2),
                round(stats_complet["maximum"], 2),
                stats_complet["nb_values"]
            ],
            fill_color="white", align="left"
        )
    ),
    row=2, col=1
)

for i, capteur in enumerate(df['id_tag'].unique()):
    df_capteur = df[df['id_tag'] == capteur]
    color = custom_palette[i % len(custom_palette)]
    
    moyenne_val = stats_complet.loc[stats_complet["id_tag"] == capteur, "moyenne"].values[0]
    minimum_val = stats_complet.loc[stats_complet["id_tag"] == capteur, "minimum"].values[0]
    maximum_val = stats_complet.loc[stats_complet["id_tag"] == capteur, "maximum"].values[0]
    count_val = stats_complet.loc[stats_complet["id_tag"] == capteur, "nb_values"].values[0]
    
    df_capteur_custom = df_capteur.copy()
    df_capteur_custom["moyenne"] = moyenne_val
    df_capteur_custom["minimum"] = minimum_val
    df_capteur_custom["maximum"] = maximum_val
    df_capteur_custom["nb_values"] = count_val

    fig.add_trace(go.Scatter(
        x=df_capteur_custom["time_bucket"],
        y=df_capteur_custom["value"],
        mode="lines+markers",
        name=capteur,
        customdata=df_capteur_custom[["id_tag", "moyenne", "minimum", "maximum", "nb_values"]],
        line=dict(color=color, width=2),
        marker=dict(size=3, opacity=0.7),
        hovertemplate=
            "<b>Capteur</b> : %{customdata[0]}<br>" +
            "<b>Date</b> : %{x|%d %b %Y %H:%M}<br>" +
            "<b>Humidité</b> : %{y:.2f} %<br><br>" +
            "<b>Statistiques capteur</b><br>" +
            "Moyenne : %{customdata[1]:.2f} %<br>" +
            "Min : %{customdata[2]:.2f} %<br>" +
            "Max : %{customdata[3]:.2f} %<br>" +
            "Nb valeurs : %{customdata[4]}<extra></extra>"
    ), row=1, col=2)

stat_trace_indices = {}
for idx, stat_capteur in enumerate(stats_complet["id_tag"]):
    df_ct = df if stat_capteur == "global" else df[df["id_tag"] == stat_capteur]
    x_vals = df_ct["time_bucket"]

    color = "red" if stat_capteur == "global" else custom_palette[idx % len(custom_palette)]
    gray = "grey" if stat_capteur == "global" else color

    moyenne_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "moyenne"].values[0]
    minimum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "minimum"].values[0]
    maximum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "maximum"].values[0]

    # Moyenne
    fig.add_trace(go.Scatter(
        x=x_vals, y=[moyenne_val]*len(x_vals),
        mode="lines",
        name=f"{stat_capteur} - Moyenne",
        line=dict(dash="dash", color=color, width=1),
        visible=False,
        hovertemplate=
            "<b>Capteur</b> : " + stat_capteur + "<br>" +
            "<b>Type</b> : Moyenne<br>" +
            "<b>Valeur</b> : %{y:.2f} %<extra></extra>"
    ), row=1, col=2)

    # Minimum
    fig.add_trace(go.Scatter(
        x=x_vals, y=[minimum_val]*len(x_vals),
        mode="lines",
        name=f"{stat_capteur} - Minimum",
        line=dict(dash="dot", color=gray, width=1),
        visible=False,
        hovertemplate=
            "<b>Capteur</b> : " + stat_capteur + "<br>" +
            "<b>Type</b> : Minimum<br>" +
            "<b>Valeur</b> : %{y:.2f} %<extra></extra>"
    ), row=1, col=2)

    # Maximum
    fig.add_trace(go.Scatter(
        x=x_vals, y=[maximum_val]*len(x_vals),
        mode="lines",
        name=f"{stat_capteur} - Maximum",
        line=dict(dash="dot", color=gray, width=1),
        visible=False,
        hovertemplate=
            "<b>Capteur</b> : " + stat_capteur + "<br>" +
            "<b>Type</b> : Maximum<br>" +
            "<b>Valeur</b> : %{y:.2f} %<extra></extra>"
    ), row=1, col=2)

    stat_trace_indices[stat_capteur] = list(range(len(fig.data)-3, len(fig.data)))

nb_base_traces = len(df['id_tag'].unique()) + 2 

buttons = [
    dict(
        label="Masquer les stats",
        method="update",
        args=[{"visible": [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)}]
    )
]

for capteur in stats_complet["id_tag"]:
    visibility = [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)
    for idx in stat_trace_indices[capteur]:
        visibility[idx] = True  
    
    buttons.append(
        dict(
            label=f"Stats {capteur}",
            method="update",
            args=[{"visible": visibility}]
        )
    )

fig.update_layout(
    updatemenus=[dict(
        type="buttons",
        buttons=buttons,
        direction="left",
        x=0.428, y=1.08,
        xanchor="left",
        yanchor="bottom"
    )],
    title="Taux d'humidité par capteur avec stats interactives",
    showlegend=True,
    margin=dict(l=10, r=10, t=100, b=50),
    height=850,
    
    yaxis=dict(
        title="Humidité (%)",
        showgrid=True,
        zeroline=False,
        tickformat="f",
        ticks="inside",
        ticklen=0,
        tickcolor="black"
    ),

    xaxis=dict(
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1 h", step="hour", stepmode="backward"),
                dict(count=6, label="6 h", step="hour", stepmode="backward"),
                dict(count=1, label="1 j", step="day", stepmode="backward"),
                dict(step="all", label="Tout")
            ])
        )
    )
)

fig.show()

In [ ]:
def creer_dashboard_capteur(df, stats_complet, titre, nom_mesure, unite, colonne_valeur="value"):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import plotly.express as px
    import locale

    custom_palette = px.colors.qualitative.D3

    fig = make_subplots(
        rows=2, cols=2,
        column_widths=[0.4, 0.6],
        row_heights=[0.7, 0.3],
        specs=[
            [{"type": "table"}, {"type": "xy", "rowspan": 2}],
            [{"type": "table"}, None]
        ],
        horizontal_spacing=0.05,
        vertical_spacing=0.1
    )

    fig.add_trace(
        go.Table(
            header=dict(values=["Heure", f"{nom_mesure} ({unite})", "Capteur"],
                        fill_color="lightgrey", align="left"),
            cells=dict(values=[df["time_bucket"], df[colonne_valeur], df["id_tag"]],
                       fill_color="white", align="left")
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Table(
            header=dict(
                values=["Capteur", f"Moyenne ({unite})", f"Min ({unite})", f"Max ({unite})", "Nb valeurs"],
                fill_color="lightgrey", align="left"
            ),
            cells=dict(
                values=[
                    stats_complet["id_tag"],
                    round(stats_complet["moyenne"], 2),
                    round(stats_complet["minimum"], 2),
                    round(stats_complet["maximum"], 2),
                    stats_complet["nb_values"]
                ],
                fill_color="white", align="left"
            )
        ),
        row=2, col=1
    )

    for i, capteur in enumerate(df["id_tag"].unique()):
        df_capteur = df[df["id_tag"] == capteur]
        color = custom_palette[i % len(custom_palette)]

        stats = stats_complet[stats_complet["id_tag"] == capteur]
        moyenne_val = stats["moyenne"].values[0]
        minimum_val = stats["minimum"].values[0]
        maximum_val = stats["maximum"].values[0]
        count_val = stats["nb_values"].values[0]

        df_capteur_custom = df_capteur.copy()
        df_capteur_custom["moyenne"] = moyenne_val
        df_capteur_custom["minimum"] = minimum_val
        df_capteur_custom["maximum"] = maximum_val
        df_capteur_custom["nb_values"] = count_val

        fig.add_trace(go.Scatter(
            x=df_capteur_custom["time_bucket"],
            y=df_capteur_custom[colonne_valeur],
            mode="lines+markers",
            name=str(capteur),
            line_shape="spline",
            customdata=df_capteur_custom[["id_tag", "moyenne", "minimum", "maximum", "nb_values"]],
            line=dict(color=color, width=2),
            marker=dict(size=3, opacity=0.7),
            hovertemplate=
                "<b>Capteur</b> : %{customdata[0]}<br>" +
                "<b>Date</b> : %{x|%d %b %Y %H:%M}<br>" +
                "<b>" + nom_mesure + "</b> : %{y:.2f} " + unite + "<br><br>" +
                "<b>Statistiques capteur</b><br>" +
                "Moyenne : %{customdata[1]:.2f} " + unite + "<br>" +
                "Min : %{customdata[2]:.2f} " + unite + "<br>" +
                "Max : %{customdata[3]:.2f} " + unite + "<br>" +
                "Nb valeurs : %{customdata[4]}<extra></extra>"
        ), row=1, col=2)

    stat_trace_indices = {}
    for idx, stat_capteur in enumerate(stats_complet["id_tag"]):
        df_ct = df if stat_capteur in ["Tous les capteurs", "global"] else df[df["id_tag"] == stat_capteur]
        x_vals = df_ct["time_bucket"]

        color = "red" if stat_capteur in ["Tous les capteurs", "global"] else custom_palette[idx % len(custom_palette)]
        gray = "grey" if stat_capteur in ["Tous les capteurs", "global"] else color

        moyenne_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "moyenne"].values[0]
        minimum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "minimum"].values[0]
        maximum_val = stats_complet.loc[stats_complet["id_tag"] == stat_capteur, "maximum"].values[0]

        for label, val, dash in zip(["Moyenne", "Minimum", "Maximum"],
                                    [moyenne_val, minimum_val, maximum_val],
                                    ["dash", "dot", "dot"]):
            fig.add_trace(go.Scatter(
                x=x_vals, y=[val]*len(x_vals),
                mode="lines",
                name=f"{stat_capteur} - {label}",
                line=dict(dash=dash, color=color, width=1),
                visible=False,
                hovertemplate=(
                    f"<b>Capteur</b> : {stat_capteur}<br>"
                    f"<b>Type</b> : {label}<br>"
                    "<b>Valeur</b> : %{y:.2f} " + f"{unite}<extra></extra>"
                )
            ), row=1, col=2)

        stat_trace_indices[stat_capteur] = list(range(len(fig.data)-3, len(fig.data)))

    nb_base_traces = len(df["id_tag"].unique()) + 2
    buttons = [
        dict(
            label="Masquer les stats",
            method="update",
            args=[{"visible": [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)}]
        )
    ]

    for capteur in stats_complet["id_tag"]:
        visibility = [True]*nb_base_traces + [False]*(len(fig.data) - nb_base_traces)
        for idx in stat_trace_indices[capteur]:
            visibility[idx] = True
        buttons.append(dict(
            label=f"Stats {capteur}",
            method="update",
            args=[{"visible": visibility}]
        ))

    # 📐 Layout
    fig.update_layout(
        updatemenus=[dict(
            type="buttons",
            buttons=buttons,
            direction="left",
            x=0.428, y=1.08,
            xanchor="left",
            yanchor="bottom"
        )],
        title=titre,
        showlegend=True,
        margin=dict(l=10, r=10, t=100, b=50),
        height=850,
        yaxis=dict(
            title=f"{nom_mesure} ({unite})",
            showgrid=True,
            zeroline=False,
            tickformat="f",
            ticks="inside",
            ticklen=0,
            tickcolor="black"
        ),
        xaxis=dict(
            rangeslider=dict(visible=True),
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1 h", step="hour", stepmode="backward"),
                    dict(count=6, label="6 h", step="hour", stepmode="backward"),
                    dict(count=1, label="1 j", step="day", stepmode="backward"),
                    dict(step="all", label="Tout")
                ])
            )
        )
    )

    fig.show()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

db_user = "admin"
db_password = "Changeme!1"
db_host = "localhost"
db_port = "5432"
db_name = "recorded"

connection_url = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

engine = create_engine(connection_url)

In [ ]:
import pandas as pd
from sqlalchemy import inspect

inspector = inspect(engine)
tables = [
    t for t in inspector.get_table_names()
    if 'source_address' in [col['name'] for col in inspector.get_columns(t)]
]

union_sql = "\nUNION ALL\n".join(
    [f"SELECT source_address AS id_tag FROM {t}" for t in tables]
)
full_query = f"SELECT DISTINCT id_tag FROM (\n{union_sql}\n) AS all_sources ORDER BY id_tag;"

# Charger le résultat dans pandas
df_capteurs = pd.read_sql(full_query, engine)
df_capteurs.head()

In [ ]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_humidity.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  sensor_humidity.humidity AS value,
  sensor_humidity.source_address AS id_tag
FROM public.sensor_humidity
GROUP BY time_bucket, sensor_humidity.humidity, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.head(10))
display(stats_complet)

In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi humidité", "Humidité", "%")

In [ ]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_pressure.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  (sensor_pressure.atmospheric_pressure / 100 ) AS value,
  sensor_pressure.source_address AS id_tag
FROM public.sensor_pressure
GROUP BY time_bucket, sensor_pressure.atmospheric_pressure, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.head(10))
display(stats_complet)

In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi pression", "pression", "hpa")

In [ ]:
import pandas as pd
from IPython.display import display

query = """
SELECT
  date_trunc('second', sensor_temperature.time AT TIME ZONE 'Europe/Paris') AS time_bucket,
  sensor_temperature.temperature AS value,
  sensor_temperature.source_address AS id_tag
FROM public.sensor_temperature
GROUP BY time_bucket, sensor_temperature.temperature, source_address
ORDER BY time_bucket ASC;
"""
df = pd.read_sql(query, engine)

stats_par_capteur = (
    df.groupby("id_tag")["value"]
      .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
      .reset_index()
)

stats_globales = (
    df["value"]
    .agg(moyenne="mean", minimum="min", maximum="max", nb_values="count")
    .to_frame().T
)
stats_globales["id_tag"] = "Tous les capteurs"

stats_complet = pd.concat([stats_par_capteur, stats_globales], ignore_index=True)

display(df.head(10))
display(stats_complet)

In [ ]:
creer_dashboard_capteur(df, stats_complet, "Suivi temperature", "temperature", "°C")